In [4]:
!pip install langchain langchain-community einops tiktoken faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.0 MB/s eta 0:00:00:00:0100:01


In [6]:
!python -m spacy download en_core_web_sm
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 90.0 MB/s eta 0:00:00:00:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 80.2 MB/s eta 0:00:00:00:01:01
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
!pip install ir_datasets deep_translator

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 16.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.0 MB/s eta 0:00:00
  Created wheel for warc3-wet-clueweb09: filename=warc3_wet_clueweb09-0.2.5-py3-none-any.whl size=18920 sha256=6d4a8bca8127871f1783e5f955fa542c2c257fb8eb299ca88fd811f0bf78372c
  Stored in directory: /root/.cache/pip/wheels/1a/d7/91/7ffb991df87e62355d945745035470ba2616aa3d83a250b5f9
  Created wheel for cbor: filename=cbor-1.0.0-cp310-cp310-linux_x86_64.whl size=53429 sha256=83eaa2bd172fe29d6f9286088bc397d1c2d50f3b938c3bc5175ccdf2bf383a29
  Stored in directory: /root/.cache/pip/wheels/85/df/c9/b39e40eccaf76dbd218556

In [7]:
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import FAISS
import faiss
import torch
from langchain.text_splitter import CharacterTextSplitter
import spacy
import pandas as pd
import numpy as np
import concurrent.futures
import pickle
from transformers import pipeline
from scipy.stats import skew
from langchain_text_splitters import TokenTextSplitter


pd.set_option('display.max_colwidth', 100)
nlp = spacy.load("es_core_news_sm")
nlp = spacy.load("en_core_web_sm")
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(torch.cuda.current_device()))

Tesla T4


In [8]:
model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/554 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.41k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:
import ir_datasets
dataset = ir_datasets.load("codec/history")

In [15]:
with open("/kaggle/input/docs-history/docs-codec-history.pkl", "rb") as f:
    history_docs = pickle.load(f)
len(history_docs)

1884

In [16]:
class TranslatedCodecDoc:
    def __init__(self, doc, translated_text: str) -> None:
        self.doc_id = doc.doc_id
        self.text = translated_text
        self.title = doc.title
        self.count = doc.count
        self.index = doc.index
        self.url = doc.url

    @classmethod
    def from_codec_doc(cls, doc, text_translated: str) -> "TranslatedCodecDoc":
        return cls(doc, text_translated)

In [17]:
from deep_translator import GoogleTranslator
translator = GoogleTranslator(source='en', target='es')

In [21]:
translated_history_docs: list[TranslatedCodecDoc] = []
for i, doc in enumerate(history_docs):
    try:
        doc_spacy = nlp(doc.text)
        new_text = ""
        current_sentences = ""
        for sent in doc_spacy.sents:
            if len(current_sentences) + len(sent.text) > 4500:
                translated_sentences = translator.translate(current_sentences)
                current_sentences = sent.text + " "
                new_text += translated_sentences + " "
            else:
                current_sentences += sent.text + " "

        translated_sentences = translator.translate(current_sentences)
        new_text += translated_sentences + " "
        current_sentences = ""

        translated_history_docs.append(TranslatedCodecDoc.from_codec_doc(doc, new_text))
        print(f"{i+1}/{len(history_docs)}: {doc.doc_id} translated")
    except Exception as e:
        print(f"Error translating doc-{i}/{doc.doc_id}: {e}")

1/1884: fe51ac49fe4a63958d32ef1e3b886d49 translated
2/1884: e63a945f0da49a22c0537507c16f6616 translated
3/1884: 2f5e71748b3537689aea1fb1e3cb763c translated
4/1884: c46ad43a9e6cd782c8d1006a07a6f462 translated
5/1884: b66eb148ac2521388b803a53505cce7c translated
6/1884: 3174f6f610ccd2068a79236909c0dfeb translated
7/1884: e58c5564c22c62e94b08dd9d20c6a3f9 translated
8/1884: 161efcacfa7b7c10e5650a49b4577398 translated
9/1884: 96b732328b98412b5f3cfe80d33af330 translated
10/1884: 02dd58fea724209b9f09ef8a8099f2fd translated
11/1884: 399c304872554fdb9b1868dcfb25a1f4 translated
12/1884: f0e5ca320db2d2992e8c84783bb610c9 translated
13/1884: abb2670b782f266b012ee2fb9f8ddf27 translated
14/1884: 5e5ab991dc3eb477772589e805dd4a1e translated
15/1884: df355131eaaf0220151dabbe80b99cb2 translated
16/1884: 5d9cb89721e7473ae05a65296884049c translated
17/1884: a46365f0cbcb09118f0c9381c37bb93b translated
18/1884: bbc4a382e6fd285b10ff69e5de1b7887 translated
19/1884: cf84b5649f82940b80112e2607938343 translated
20

In [30]:
doc_missing = None
for doc in history_docs:
    if doc.doc_id == "ce1488b41c49608bf6221f13909da46a":
        doc_missing = doc
        print(doc.text)
        break

The arts
In its literature, England arguably has attained its most influential cultural expression. For more than a millennium, each stage in the development of the English language has produced its masterworks.
Little is known of English literature before the arrival of the Anglo-Saxons, though echoes of England’s Celtic past resound in Arthurian legend. Anglo-Saxon literature, written in the Old English language, is remarkably diverse. Its surviving corpus includes hymns, lyric poems such as “The Wanderer” and “The Seafarer,” riddles and spells, songs, and the epic poem Beowulf, which dates from the 9th or 10th century. Following the Norman Conquest of 1066, French influence shaped the vocabulary as well as the literary preoccupations of Middle English. Geoffrey Chaucer epitomized both the courtly philosophical concerns and the earthy vernacular of this period in his Troilus and Criseyde and The Canterbury Tales, respectively, while William Langland’s Piers Plowman was an early expre

In [31]:
translated_doc_missing = translated_history_docs.append(TranslatedCodecDoc.from_codec_doc(doc_missing, """Las artes
En su literatura, Inglaterra ha alcanzado posiblemente su expresión cultural más influyente. Durante más de un milenio, cada etapa del desarrollo del inglés ha producido sus obras maestras.
Se sabe poco de la literatura inglesa anterior a la llegada de los anglosajones, aunque resuenan ecos del pasado celta de Inglaterra en la leyenda artúrica. La literatura anglosajona, escrita en inglés antiguo, es notablemente diversa. Su corpus superviviente incluye himnos, poemas líricos como «El vagabundo» y «El navegante», acertijos y conjuros, canciones y el poema épico Beowulf, que data del siglo IX o X. Tras la conquista normanda de 1066, la influencia francesa moldeó el vocabulario y las inquietudes literarias del inglés medio. Geoffrey Chaucer personificó tanto las inquietudes filosóficas cortesanas como la mundana lengua vernácula de este período en sus obras Troilo y Crésida y Los cuentos de Canterbury, respectivamente, mientras que Piers Plowman, de William Langland, fue una expresión temprana de la disidencia religiosa y política que posteriormente caracterizaría la literatura inglesa. La era isabelina de finales del siglo XVI impulsó el florecimiento del Renacimiento europeo en Inglaterra y la edad de oro de la literatura inglesa. Las obras de William Shakespeare, si bien representan en apariencia la culminación del inglés isabelino, alcanzan una profundidad de caracterización y una riqueza de invención que las han consolidado en el repertorio dramático de prácticamente todas las lenguas. La publicación de la versión King James de la Biblia en 1611 infundió en la literatura de la época imágenes religiosas y un lenguaje notablemente vigoroso, y sirvió como un importante instrumento para la difusión de la alfabetización en toda Inglaterra. Los conflictos políticos y religiosos del siglo XVII sirvieron de telón de fondo para una prolífica poesía, que abarca desde las introspecciones metafísicas de John Donne hasta las épicas visionarias de John Milton, además de la alegoría en prosa El progreso del peregrino de John Bunyan.
La dicotomía entre clasicismo y romanticismo, así como entre razón e imaginación, llegó a dominar el siglo XVIII, con la sátira y la crítica neoclásicas de Alexander Pope, Jonathan Swift y Samuel Johnson, por un lado, y la autoexpresión romántica, algo posterior, de William Blake, William Wordsworth, Samuel Taylor Coleridge y John Keats, por otro. También durante este período, la novela emergió como una forma capaz de incorporar la vida cotidiana al ámbito literario, como se puede apreciar en la obra de Jane Austen. Aproximadamente en esta época, las distintas regiones de Inglaterra comenzaron a ejercer una poderosa influencia en muchos escritores, como el Distrito de los Lagos en Wordsworth, los páramos de Yorkshire en las hermanas Brontë (Anne, Charlotte y Emily), Dorset en Thomas Hardy, las cuencas mineras de Midlands en D.H. Lawrence y Londres en Charles Dickens. A mediados y finales del siglo XIX, la literatura inglesa abordó cada vez más las preocupaciones sociales, dando lugar a los escritos utópicos de William Morris y Samuel Butler, el análisis psicológico de George Eliot, las novelas realistas de Elizabeth Gaskell y las historias y fábulas nacionalistas de Rudyard Kipling. Muchos escritores también encontraron un nuevo público en los niños, dando lugar a obras como Alicia en el país de las maravillas de Lewis Carroll y generando clásicos posteriores como El viento en los sauces de Kenneth Grahame, los cuentos de Peter Rabbit de Beatrix Potter, Winnie-the-Pooh de A.A. Milne, J.R.R. El Hobbit de Tolkien, e incluso, podría decirse, la obra de finales del siglo XX de J.K. Rowling.
La literatura inglesa del siglo XX fue reinterpretada por escritores nativos como Virginia Woolf. También absorbió y transmutó elementos foráneos, incorporando a la corriente principal de su tradición a poetas tan irlandeses como William Butler Yeats, tan galeses como Dylan Thomas, o tan arraigados en la línea clásica como los expatriados estadounidenses T.S. Eliot y Henry James. Novelistas populares como Agatha Christie, P.D. James, Dick Francis y John Le Carré alimentaron la afición inglesa por las novelas de misterio y los procedimientos policiales, mientras que los poetas W.H. Auden, Ted Hughes y Philip Larkin aportaron un nuevo enfoque a las cuestiones de las relaciones personales, y los novelistas Anthony Burgess, Graham Greene y Kingsley Amis abordaron ambigüedades morales y dilemas modernos. Muchos otros, como Iris Murdoch y Martin Amis, trabajaron en una línea cómica o satírica consolidada. La inmigración continuó diversificando el panorama literario inglés, dando lugar a escritores como V.S. Naipaul, Salman Rushdie y Kazuo Ishiguro. (Para más información, véase Literatura inglesa).
"""))

In [32]:
len(translated_history_docs)

1884

In [33]:
with open("translated-docs-codec-history.pkl", "wb") as f:
    pickle.dump(translated_history_docs, f)

In [40]:
class CodecHistoryStore:
    def __init__(self, docs: list[TranslatedCodecDoc]) -> None:
        self.docs = docs
        self.chunks = []
        self.chunks_ids = []
        self.embeddings = None

        self.__create_chunks()

    def __create_chunks(self):
        text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=50)
        for i, doc in enumerate(self.docs):
            new_chunks = text_splitter.split_text(doc.text)
            self.chunks.extend(new_chunks)
            self.chunks_ids.extend([i] * len(new_chunks))
    
    def doc_from_chunk_id(self, chunk_id: int) -> TranslatedCodecDoc:
        return self.docs[self.chunks_ids[chunk_id]]

    def generate_embeddings(self):
        model = SentenceTransformer("nomic-ai/nomic-embed-text-v2-moe", trust_remote_code=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)

        self.embeddings = model.encode(store.chunks, convert_to_tensor=True, prompt_name="passage")

In [41]:
store = CodecHistoryStore(translated_history_docs)
len(store.chunks), len(store.chunks_ids)

(12707, 12707)

In [42]:
store.generate_embeddings()
store.embeddings.shape

Batches:   0%|          | 0/398 [00:00<?, ?it/s]

torch.Size([12707, 768])

In [43]:
with open("store-with-embeddings-translated-docs-codec-history.pkl", "wb") as f:
    pickle.dump(store, f)